In [74]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np 
import tensorflow as tf
import sqlite3
from pickle import load
from time import sleep
from sklearn.preprocessing import StandardScaler
from apikey import API_KEY

In [75]:
watcher = LolWatcher(API_KEY)
region = 'NA1'
sc = load(open('scaler.pkl', 'rb'))

In [76]:
model = tf.keras.models.load_model('model_w_params')

In [77]:
def get_summoner_uid(name):
    summoner = watcher.summoner.by_name(region, name)
    return summoner["puuid"]

In [78]:
ign = "Linom"
# puid = get_summoner_uid(ign)
puid = "pXozCpOspMGJvjixjONFRy6dBG9bj5rt4KeJOJCr1wAGprTEDaNOX3OTFgG6o8RK1QpFjihDLiIVxg"

In [120]:
def get_data(player_uid): 
    
    match_list = watcher.match.matchlist_by_puuid('americas', player_uid, type="ranked", count=1)


    game_timeline = watcher.match.timeline_by_match('americas', match_list[0])
    game_details = watcher.match.by_id('americas', match_list[0])
    if game_details["info"]["participants"][0]["win"] == True:
        winner = "Team 1"
    else:
        winner = "Team 2"


    if len(game_timeline['info']['frames']) < 22:
        # print("less than 20")
        return("Game not long enough to predict")
    # print("more than 20")
    frames =  game_timeline['info']['frames'][:22]

    # get gold diff 
    gold = {}
    for k in range(len(frames)):
        gold['{}'.format(k)] = {}
    for value in gold.values():
        for l in range(1, 11):
            value['{}'.format(l)] = 0

    for m in range(len(frames)):
        for n in range(1, 11):
            gold['{}'.format(m)]['{}'.format(n)] = game_timeline['info']['frames'][m]['participantFrames']['{}'.format(n)]['totalGold']

    blue_gold = []
    red_gold = []

    for key in gold.keys():
        team1 = 0
        team2 = 0
        for o in range(1, 6):
            team1 += gold[key]['{}'.format(o)]
            team2 += gold[key]['{}'.format(o + 5)]
        blue_gold.append(team1)
        red_gold.append(team2)

    # gold_diff = (np.array(team_1_gold) - np.array(team_2_gold)).tolist()


    blue_dragons = 0
    blue_heralds = 0
    red_dragons = 0
    red_heralds = 0
    blue_barons = 0
    red_barons = 0

    for frame in frames:
        for event in frame["events"]:
            if "monsterType" in event.keys():
                if event["monsterType"] == "DRAGON":
                    if event["killerTeamId"] == 100:
                        blue_dragons += 1
                    else:
                        red_dragons += 1
                if event["monsterType"] == "RIFTHERALD":
                    if event["killerTeamId"] == 100:
                        blue_heralds += 1
                    else:
                        red_heralds += 1
                if event["monsterType"] == "BARON_NASHOR":
                    if event["killerTeamId"] == 100:
                        blue_barons += 1
                    else:
                        red_barons += 1

    # print(match, blue_gold[-1], red_gold[-1], blue_dragons, blue_heralds, blue_barons, red_dragons, red_heralds, red_barons)

    blue_k = 0 # blue side (bottom)
    red_k = 0 # red side (top)

    for frame in frames:
        for event in frame["events"]:
            if "victimId" in event.keys():
                if event["type"] == "CHAMPION_KILL":
                    if event["killerId"] == 0:
                        continue
                    if 1 <= event["killerId"] <= 5:
                        blue_k += 1
                    else:
                        red_k += 1

        blue_obj = 0
        red_obj = 0
        for frame in frames:
            for event in frame["events"]:
                if "buildingType" in event.keys():
                    if event["teamId"] == 100:
                        red_obj += 1
                    else:
                        blue_obj += 1
                        

    tensor = [blue_dragons, blue_heralds, blue_barons, blue_obj, blue_k, blue_gold[-1], red_dragons, red_heralds, red_barons, red_obj, red_k, red_gold[-1]]
    
    return tensor,winner

### Use our pickled scaling weights to predict new raw data

In [121]:
result = get_data(puid)
input_data = result[0]
winner = result[1]

input_data
input_data = sc.transform([input_data])
input_data

array([[-1.25542406,  0.09737731, -0.26427777, -0.86774857,  1.34946934,
         0.02503562,  1.57446738,  0.08439657, -0.27851056,  0.04976615,
         0.12134288,  0.12986495]])

<0.5 = Team 2, else Team 1

In [122]:
model.predict(input_data)

array([[0.3206097]], dtype=float32)

In [123]:
print(winner)

Team 2
